Okay, to start off I need to import pandas, the "import pandas as pd" lets me us pd as shorthand when I call on pandas

In [1]:
import pandas as pd


Next I add in my datasets. I'll start with just 1 set, the calendar, and print the begining of the dataset to make sure its reading correctly

In [2]:
calendar = pd.read_csv('Datasets\calendar.csv')
print(calendar.shape)
calendar.head(5)

(358, 7)


,date,day,event,holiday,last_of_the_month,popup,sport
0,11/01/2022,tuesday,NaN,NaN,NaN,NaN,NaN
1,11/02/2022,wednesday,Karaoke,NaN,NaN,NaN,NaN
2,11/03/2022,thursday,NaN,NaN,NaN,NaN,NaN
3,11/04/2022,friday,NaN,NaN,NaN,NaN,NaN
4,11/05/2022,saturday,NaN,NaN,NaN,NaN,NaN


(The NaN means not a number, just any spot left empty for no drinks sold)

Now the other datasets

In [3]:
normal_sales = pd.read_csv('Datasets/MOCK_BAR_DATA.csv')
halloween = pd.read_csv('Datasets/MOCK_BAR_DATA_HALLOWEEN.csv')

I've seperated the mock sales into 2 sets for the sake of making this project easier to understand, by using the week of the halloween popup as a shortened small scale example before I break apart the rest of the year. (I actually used more than a year, from Nov 1,2022 through Dec 31,2023) 

In [4]:
halloween

,date,holiday_drink_1,holiday_drink_2,holiday_drink_3,holiday_drink_4,holiday_drink_5,holiday_shot,holiday_bowl,draft_beer,can_beer
0,10/24/2023,2,6,4,7,2,5,NaN,NaN,NaN
1,10/25/2023,2,10,15,3,5,4,NaN,NaN,NaN
2,10/26/2023,10,5,11,3,9,2,NaN,9.0,5.0
3,10/27/2023,8,10,2,6,3,1,3.0,6.0,NaN
4,10/28/2023,18,14,12,15,14,14,5.0,12.0,NaN
5,10/29/2023,6,2,9,7,1,1,3.0,14.0,NaN
6,10/31/2023,13,14,3,7,8,3,4.0,7.0,NaN


Now to combine the calendar to the datasets to set up my correlations.

In [5]:
halloween_popup = halloween.set_index("date").join(calendar.set_index("date"))
sales = normal_sales.set_index("date").join(calendar.set_index("date"))

And again I will check that they were combined right, using Halloween as an example. (I'll use the head() syntax to keep it short enough for the notebook for the example)

In [6]:
halloween_popup.head (3)

,holiday_drink_1,holiday_drink_2,holiday_drink_3,holiday_drink_4,holiday_drink_5,holiday_shot,holiday_bowl,draft_beer,can_beer,day,event,holiday,last_of_the_month,popup,sport
date,,,,,,,,,,,,,,,
10/24/2023,2,6,4,7,2,5,NaN,NaN,NaN,tuesday,NaN,NaN,NaN,Halloween,NaN
10/25/2023,2,10,15,3,5,4,NaN,NaN,NaN,wednesday,Karaoke,NaN,NaN,Halloween,NaN
10/26/2023,10,5,11,3,9,2,NaN,9.0,5.0,thursday,Emo,NaN,NaN,Halloween,NaN


With the example shoing it combined the Halloween sales with the calendar, we can see some data columns aren't necissary for every set we are correlating. This set is the easiest to see which ones we need, as its only really looking at the effects of a single holiday popup. So I can drop the sport, last_of_the_month, and popup columns. The lst of the month column refers to something all year round, and I don't need to track sports during a popup, and I already know it's a popup.

I also will remove the NaN to make it a little easier to read.

In [7]:
halloween_popup = halloween_popup.drop(columns=['last_of_the_month', 'popup', 'sport'])
halloween_popup = halloween_popup.fillna(' ')
halloween_popup

,holiday_drink_1,holiday_drink_2,holiday_drink_3,holiday_drink_4,holiday_drink_5,holiday_shot,holiday_bowl,draft_beer,can_beer,day,event,holiday
date,,,,,,,,,,,,
10/24/2023,2,6,4,7,2,5,,,,tuesday,,
10/25/2023,2,10,15,3,5,4,,,,wednesday,Karaoke,
10/26/2023,10,5,11,3,9,2,,9.0,5.0,thursday,Emo,
10/27/2023,8,10,2,6,3,1,3.0,6.0,,friday,,
10/28/2023,18,14,12,15,14,14,5.0,12.0,,saturday,,Halloween Party
10/29/2023,6,2,9,7,1,1,3.0,14.0,,sunday,,
10/31/2023,13,14,3,7,8,3,4.0,7.0,,tuesday,,Halloween


Even without a graph, you can see some details in these sales that point towards some of the correlations I'm looking for. Earlier in the week, there arent many bowl drinks ordered. Emo night is the only night during the popup anyone ordered canned beer, mostly cheap beers to go with shots. (I chose to simplify draft and canned beer, as this fake data is based off a cocktail bar that doesnt sell much beer). The saturday night of the halloween party had higher sales than the rest of the week. compared to teh rest of the year, theres a much higher ammount of draft sales the week of halloween because the bar partnered with a brewery to make candy themed beers that got more attention from the people who drink there.

For the full year of sales, I'll  seperate the data into multiple dataframes to check if the correlations I'm expecting are there. Then I will save those dataframes into their own csv's to make graphs, because the patterns wont stand out as much with a dataframe with 350 rows of numbers